# Predicción de severidad de accidentes de tránsito



1.   Tenemos un archivo con datos: train.csv el cual contiene información sobre accidentes de tránsito en EEUU y un archivo test4alumnxs.csv que tiene la misma info pero sin las etiquetas para Severity (más información abajo).
2.   El objetivo es usar los datos para construir un modelo que prediga la severidad del accidente (columna 'Severity', donde 0 es baja severidad y 1 es alta severidad). Para esto, primero puedo usar los datos del archivo train.csv para ajustar los parámetros del modelo, ajustar los hiperparámetros, combinar los features para agregar nuevos, estandarizar los datos, etc. Al terminar esto, tendré un modelo entrenado en estos datos, y una idea de que tan bien funciona. Luego generaremos un vector de probabilidades sobre el test4alumnos.csv para tratar de predecir la severidad del accidente.

# Los datos
Los datos corresponden a registros de accidentes de tránsito en EEUU entre 2016 y 2021.

Las primeras dos columnas ("unnamed") no sirve nara nada y se pueden descartar. 

La columna 'Severity' contiene la severidad del accidente (0 significa baja, 1 significa alta)

Luego tenemos la hora de comienzo y final del accidente, la latitud/longitud inicial y final del accidente (entre que valores queda delimitado), la distancia de camino afectada por el accidente, una descripción verbal hecha por un humano, número y nombre de la calle donde ocurrió, lado de la calle (izquierda o derecha), ciudad, condado, estado, código postal, huso horario, código de aeropuerto donde se encuentra la estación meterológica más cercana al accidente, hora en la cual se midió el clima,temperatura, temperatura del viento, humedad, presión, visibilidad, dirección del viento, condición climática (despejado, sol, nieve, etc), presencia de comodidades ("amenities") en la cercanía del accidente, presencia de loma de burro, presencia de un cruce, presencia de señal de ceder paso, presencia de unión de calles, presencia de cartel de no salida, presencia de vías de tren, presencia de rotonda, presencia de estación, presencia de signo de parar, presencia de moderadores de tráfico, presencia de señales de tránsito, presencia de calle en forma de U ("loop"), día u oscuridad de acuerdo a la salida del sol, día u oscuridad de acuerdo a la penumbra civil (si es o no necesario utilizar alumbramiento eléctrico), día u oscuridad de acuerdo a la penumbra náutica, día u oscuridad de acuerdo a la penumbra astronómica.

No todos estos campos son necesariamente útiles, y tampoco no todos están en un formato numérico directamente utilizable (por ejemplo, hay strings en la parte de descripción del accidente). Será parte del trabajo decidir que features quedarse, que features sumar, y como extraer información relevante de los features más complejos. 

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

drive.mount('/content/drive') # montamos el drive

# cargamos el dataframe de entrenamiento y el dataframe de testeo
# el dataframe de entrenamiento tiene toda la información, incluyendo la columna 'Severity' (severidad)
# esta es la columna que vamos a entrenar el modelo para predecir: 0=severidad baja; 1=severidad alta.
# en cambio, el dataframe de test no tiene esta columna: son los datos que usamos para generar la predicción que se entrega 
filename_train = '/content/drive/My Drive/LaboDatos2022/SegundaEjercitacion/train.csv' 
filename_test = '/content/drive/My Drive/LaboDatos2022/SegundaEjercitacion/test4alumnxs.csv' 

df_train = pd.read_csv(filename_train)
df_test = pd.read_csv(filename_test)
df_test.columns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Index(['Unnamed: 0', 'Unnamed: 0.1', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Number', 'Street', 'Side', 'City', 'County', 'State', 'Zipcode',
       'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [ ]:
df_train.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [ ]:
df_train["Severity"].value_counts()

0    28072
1    17762
Name: Severity, dtype: int64

In [1]:
# Como el dataset no parece tener un desbalance importante, sigo adelante.

In [ ]:
df_train["Weather_Condition"].value_counts()

Fair                           21874
Cloudy                          7202
Mostly Cloudy                   5689
Partly Cloudy                   3998
Light Rain                      2103
Fog                              871
Light Snow                       809
Rain                             420
Haze                             419
Fair / Windy                     344
Overcast                         219
Smoke                            199
Heavy Rain                       175
Thunder in the Vicinity          156
Cloudy / Windy                   137
Light Drizzle                    128
T-Storm                          124
Mostly Cloudy / Windy            114
Snow                             113
Thunder                          110
Light Rain with Thunder           95
Wintry Mix                        74
Light Rain / Windy                69
Partly Cloudy / Windy             69
Heavy T-Storm                     56
Drizzle                           46
Light Snow / Windy                32
H

In [ ]:
# Agrego algunas columnas a train.csv y a test4alumnxs.csv para testear mis hipótesis:
# 1) Los accidentes más severos son los que se producen en autopistas o carreteras de alta velocidad (por ejemplo, rutas nacionales).
# 2) Los accidentes más severos contienen una advertencia del tipo "blocked", "multi" o "closed road" (u otras)
# 3) Los accidentes más severos se producen de noche.
# 4) Los accidentes más severos se producen en condiciones climatológicas desfavorables (lluvia, nieve, niebla, etc).
def highway_or_road(a):
    if "highway" in a.lower():
        return True
    elif "hwy" in a.lower():
        return True
    elif "road" in a.lower():
      return True
    elif "rd" in a.lower():
      return True
    elif "route" in a.lower():
      return True
    else:
        return False
df_train["Highway_or_road"] = df_train["Street"].apply(lambda x: highway_or_road(x))
df_test["Highway_or_road"] = df_test["Street"].apply(lambda x: highway_or_road(x))

def serious(a):
    #if "drive with caution" in a.lower():
        #return True
    if "closed" in a.lower():
        return True
    elif "blocked" in a.lower():
      return True
    elif "closure" in a.lower():
      return True
    elif "multi" in a.lower():
      return True
    elif "serious" in a.lower():
      return True
    else:
        return False
df_train["Serious"] = df_train["Description"].apply(lambda x: serious(x))
df_test["Serious"] = df_test["Description"].apply(lambda x: serious(x))

def night(a):
  if "Night" in a.lower():
    return True
  else:
    return False

df_train["Night"] = df_train["Civil_Twilight"].apply(lambda x: night(x))
df_test["Night"] = df_test["Civil_Twilight"].apply(lambda x: night(x))

def adverse_weather(a):
  if "rain" in a.lower():
    return True
  elif "snow" in a.lower():
    return True
  elif "fog" in a.lower():
    return True
  elif "smoke" in a.lower():
    return True
  else:
    return False

df_train["Adverse_Weather"] = df_train["Weather_Condition"].apply(lambda x: adverse_weather(x))
df_test["Adverse_Weather"] = df_test["Weather_Condition"].apply(lambda x: adverse_weather(x))

df_train.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight', 'Highway_or_road', 'Serious', 'Night',
       'Adverse_Weather'],
      dtype='object')

In [ ]:
df_test.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Number', 'Street', 'Side', 'City', 'County', 'State', 'Zipcode',
       'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight', 'Highway_or_road', 'Serious', 'Night',
       'Adverse_Weather'],
      dtype='object')

In [ ]:
df_test.shape

(11372, 51)

In [ ]:
df_train

,Unnamed: 0,Unnamed: 0.1,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Highway_or_road,Serious,Night,Adverse_Weather
0,2,2,0,2021-08-19 23:58:30.000000000,2021-08-20 00:36:30.000000000,35.879641,-78.734582,35.884381,-78.740822,0.479,...,False,False,Night,Night,Night,Night,False,False,False,True
1,7,7,0,2021-01-28 21:09:00.000000000,2021-01-29 01:18:47.000000000,36.206389,-121.116768,36.206635,-121.116980,0.021,...,False,False,Night,Night,Night,Night,False,False,False,True
2,11,11,0,2021-02-12 04:48:00,2021-02-12 06:26:21,30.499754,-91.138325,30.499795,-91.136996,0.079,...,False,False,Night,Night,Night,Night,False,False,False,True
3,12,12,0,2020-12-09 13:25:30,2020-12-10 00:19:49,26.548679,-81.871670,26.546193,-81.871104,0.175,...,False,False,Day,Day,Day,Day,False,False,False,False
4,13,13,0,2020-10-30 16:30:18,2020-10-30 17:45:26,45.533651,-94.100321,45.532940,-94.099488,0.064,...,False,False,Day,Day,Day,Day,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45829,159985,159985,1,2020-05-11 16:44:04,2020-05-11 17:12:48,35.656480,-118.444170,35.656830,-118.440450,0.210,...,False,False,Day,Day,Day,Day,True,True,False,False
45830,159987,159987,1,2018-12-05 03:57:02,2018-12-05 04:26:27,39.492480,-84.131800,39.517181,-84.098369,2.468,...,False,False,Night,Night,Night,Night,True,True,False,True
45831,159991,159991,1,2020-12-02 03:13:00,2020-12-02 08:05:55,41.158851,-79.396466,41.172221,-79.399466,0.937,...,False,False,Night,Night,Night,Night,True,True,False,False
45832,159992,159992,1,2019-09-10 10:16:56,2019-09-10 10:45:23,30.287466,-81.720190,30.288360,-81.720190,0.062,...,False,False,Day,Day,Day,Day,False,True,False,False


Luego, sacamos el vector de etiquetas (severidad).

En este modelo vamos a usar el subconjunto de features que corresponde a variables climáticas: temperatura, temperatura del viento, humedad, presión, visibilidad, velocidad del viento y precipitación así como otras correspondientes al contexto del accidente (si era un cruce de calles, si ocurrió en autopista, si el clima era adverso, etc.).

Con esto armamos la matriz X. Luego, separamos un 70%/30% para train y test del modelo.



In [ ]:
y = np.array(df_train['Severity']) # armo el vector de etiquetas

# vamos a entrenar un modelo con estas variables
columnas = ['Temperature(F)', 'Wind_Chill(F)', 'Distance(mi)', 'Stop', 'Humidity(%)','Junction','Crossing', "Highway_or_road", "Serious", "Night", "Adverse_Weather",'Traffic_Signal', 'Traffic_Calming','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Precipitation(in)']

# extraigo la matriz de features X
X = np.array(df_train[columnas])

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)


In [ ]:
# pruebo con un random forest así como viene de fábrica
clf_rf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                             max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, 
                             random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)  

# ajusto el modelo
clf_rf.fit(X_train, y_train) #Como veo que es el mejor modelo, lo nombro de diferente manera así puedo luego armar el vector de etiquetas.

# armo el vector de predicciones.
y_hat_test = clf_rf.predict_proba(X_test)[:, 1]
y_hat_train = clf_rf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_train, roc_test

(0.9999877545112356, 0.9758308960573187)

In [ ]:
auc_values = [] # aca es donde van a ir a parar los AUCs de cada fold
scores = np.array([])     # aca es donde van a ir a parar los scores computados para todos los casos
indices = np.array([])    # aca es donde van a ir a parar los indices correspondientes a las entradas de scores
for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True).split(X, y): # va generando los indices que corresponden a train y test en cada fold
    X_train, X_test = X[train_index], X[test_index] # arma que es dato de entrenamiento y qué es dato de evaluación
    y_train, y_test = y[train_index], y[test_index]     # idem con los targets

    clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                             max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, 
                             random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None) # Inicializamos nuevamente el modelo. max_iter es la cantidad de iteraciones maximas del algoritmo de optimizacion de parametros antes de detenerse.
    clf.fit(X_train, y_train) # Ajustamos el modelo con los datos de entrenamiento

    probas_test = clf.predict_proba(X_test)  # probabilidades con datos de evaluación
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, probas_test[:,1]) # para tener curva ROC con datos de entrenamiento
    auc_test = roc_auc_score(y_test, probas_test[:,1]) #  AUC con datos de evaluación

    auc_values.append(auc_test)
    scores = np.concatenate((scores,probas_test[:,1]),axis=0)
    indices = np.concatenate((indices,test_index),axis=0)

print("Estos son los valores AUC para cada fold:")
print(auc_values)
print("Estos es el promedio de todos los AUC:")
print(np.mean(auc_values))
print("Estos son las probabilidades para cada sample:")
print(scores)
print("Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):")
print(indices)

Estos son los valores AUC para cada fold:
[0.9716992621517332, 0.9744444076757078, 0.9760936507190069, 0.9777392492392917, 0.9742788048780879]
Estos es el promedio de todos los AUC:
0.9748510749327656
Estos son las probabilidades para cada sample:
[0.02 0.02 0.   ... 0.96 0.96 1.  ]
Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):
[0.0000e+00 9.0000e+00 1.2000e+01 ... 4.5825e+04 4.5829e+04 4.5831e+04]


In [ ]:
# pruebo con una regresion logistica así como viene de fábrica
clf = LogisticRegression()  

# ajusto el modelo
clf.fit(X_train, y_train)

# armo el vector de predicciones.
y_hat_test = clf.predict_proba(X_test)[:, 1]
y_hat_train = clf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_train, roc_test

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


(0.9698694424290579, 0.9689956756275335)

In [ ]:
auc_values = [] # aca es donde van a ir a parar los AUCs de cada fold
scores = np.array([])     # aca es donde van a ir a parar los scores computados para todos los casos
indices = np.array([])    # aca es donde van a ir a parar los indices correspondientes a las entradas de scores
for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True).split(X, y): # va generando los indices que corresponden a train y test en cada fold
    X_train, X_test = X[train_index], X[test_index] # arma que es dato de entrenamiento y qué es dato de evaluación
    y_train, y_test = y[train_index], y[test_index]     # idem con los targets

    clf = clf # Inicializamos nuevamente el modelo. max_iter es la cantidad de iteraciones maximas del algoritmo de optimizacion de parametros antes de detenerse.
    clf.fit(X_train, y_train) # Ajustamos el modelo con los datos de entrenamiento

    probas_test = clf.predict_proba(X_test)  # probabilidades con datos de evaluación
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, probas_test[:,1]) # para tener curva ROC con datos de entrenamiento
    auc_test = roc_auc_score(y_test, probas_test[:,1]) #  AUC con datos de evaluación

    auc_values.append(auc_test)
    scores = np.concatenate((scores,probas_test[:,1]),axis=0)
    indices = np.concatenate((indices,test_index),axis=0)

print("Estos son los valores AUC para cada fold:")
print(auc_values)
print("Estos es el promedio de todos los AUC:")
print(np.mean(auc_values))
print("Estos son las probabilidades para cada sample:")
print(scores)
print("Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):")
print(indices)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Estos son los valores AUC para cada fold:
[0.9678783302447594, 0.9676318460045085, 0.9679762787955928, 0.9703648883099637, 0.9702855102831723]
Estos es el promedio de todos los AUC:
0.9688273707275993
Estos son las probabilidades para cada sample:
[0.00857938 0.0096776  0.00973871 ... 0.92342349 0.91992661 0.91535417]
Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):
[1.0000e+00 6.0000e+00 7.0000e+00 ... 4.5824e+04 4.5826e+04 4.5833e+04]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
# pruebo con un arbol de decision así como viene de fábrica
clf = DecisionTreeClassifier()  

# ajusto el modelo
clf.fit(X_train, y_train)

# armo el vector de predicciones.
y_hat_test = clf.predict_proba(X_test)[:, 1]
y_hat_train = clf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_train, roc_test

(0.9999906714599758, 0.9197929253844154)

In [ ]:
auc_values = [] # aca es donde van a ir a parar los AUCs de cada fold
scores = np.array([])     # aca es donde van a ir a parar los scores computados para todos los casos
indices = np.array([])    # aca es donde van a ir a parar los indices correspondientes a las entradas de scores
for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True).split(X, y): # va generando los indices que corresponden a train y test en cada fold
    X_train, X_test = X[train_index], X[test_index] # arma que es dato de entrenamiento y qué es dato de evaluación
    y_train, y_test = y[train_index], y[test_index]     # idem con los targets

    clf = clf # Inicializamos nuevamente el modelo. max_iter es la cantidad de iteraciones maximas del algoritmo de optimizacion de parametros antes de detenerse.
    clf.fit(X_train, y_train) # Ajustamos el modelo con los datos de entrenamiento

    probas_test = clf.predict_proba(X_test)  # probabilidades con datos de evaluación
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, probas_test[:,1]) # para tener curva ROC con datos de entrenamiento
    auc_test = roc_auc_score(y_test, probas_test[:,1]) #  AUC con datos de evaluación

    auc_values.append(auc_test)
    scores = np.concatenate((scores,probas_test[:,1]),axis=0)
    indices = np.concatenate((indices,test_index),axis=0)

print("Estos son los valores AUC para cada fold:")
print(auc_values)
print("Estos es el promedio de todos los AUC:")
print(np.mean(auc_values))
print("Estos son las probabilidades para cada sample:")
print(scores)
print("Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):")
print(indices)

Estos son los valores AUC para cada fold:
[0.9138342288192022, 0.918554131268401, 0.9210528822489633, 0.9184313250888301, 0.9182481376995093]
Estos es el promedio de todos los AUC:
0.9180241410249812
Estos son las probabilidades para cada sample:
[0. 0. 0. ... 1. 1. 1.]
Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):
[2.0000e+00 8.0000e+00 9.0000e+00 ... 4.5828e+04 4.5829e+04 4.5831e+04]


In [ ]:
# pruebo con una red de perceptrones multilayer así como viene de fábrica
clf = MLPClassifier()  

# ajusto el modelo
clf.fit(X_train, y_train)

# armo el vector de predicciones.
y_hat_test = clf.predict_proba(X_test)[:, 1]
y_hat_train = clf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_train, roc_test

(0.9712776023101439, 0.9724091075400302)

In [ ]:
auc_values = [] # aca es donde van a ir a parar los AUCs de cada fold
scores = np.array([])     # aca es donde van a ir a parar los scores computados para todos los casos
indices = np.array([])    # aca es donde van a ir a parar los indices correspondientes a las entradas de scores
for train_index, test_index in StratifiedKFold(n_splits=5, shuffle=True).split(X, y): # va generando los indices que corresponden a train y test en cada fold
    X_train, X_test = X[train_index], X[test_index] # arma que es dato de entrenamiento y qué es dato de evaluación
    y_train, y_test = y[train_index], y[test_index]     # idem con los targets

    clf = clf # Inicializamos nuevamente el modelo. max_iter es la cantidad de iteraciones maximas del algoritmo de optimizacion de parametros antes de detenerse.
    clf.fit(X_train, y_train) # Ajustamos el modelo con los datos de entrenamiento

    probas_test = clf.predict_proba(X_test)  # probabilidades con datos de evaluación
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, probas_test[:,1]) # para tener curva ROC con datos de entrenamiento
    auc_test = roc_auc_score(y_test, probas_test[:,1]) #  AUC con datos de evaluación

    auc_values.append(auc_test)
    scores = np.concatenate((scores,probas_test[:,1]),axis=0)
    indices = np.concatenate((indices,test_index),axis=0)

print("Estos son los valores AUC para cada fold:")
print(auc_values)
print("Estos es el promedio de todos los AUC:")
print(np.mean(auc_values))
print("Estos son las probabilidades para cada sample:")
print(scores)
print("Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):")
print(indices)

Estos son los valores AUC para cada fold:
[0.9710992715779002, 0.9699487527375997, 0.9698373281945312, 0.9731044107795728, 0.9706249127422756]
Estos es el promedio de todos los AUC:
0.970922935206376
Estos son las probabilidades para cada sample:
[0.02054257 0.01615146 0.00775284 ... 0.83811484 0.77615284 0.89220482]
Esta es la forma en que quedaron ordenados las entradas del vector anterior (indices):
[0.0000e+00 1.9000e+01 3.1000e+01 ... 4.5816e+04 4.5824e+04 4.5831e+04]


Por último, veo que el RandomForest es quien obtiene los mejores resultados. Aplicamos el modelo ya entrenado a los datos de test4alumnxs.csv y guardo el vector "predicciones" y lo descargo, tal como muestra el screenshot de más abajo.

In [ ]:
X = np.array(df_test[columnas]) # cargo el dataset de testeo

predicciones = clf_rf.predict_proba(X)[:, 1] # obtengo el vector de probabilidades
predicciones.size
# Controlo que el vector de probabilidades tenga el mismo largo que las filas de df_test


11372

In [ ]:
print(predicciones)

# lo guardo en el espacio de trabajo de colab
np.savetxt('predicciones.csv', predicciones, delimiter=",")

[0.   0.03 0.04 ... 0.01 0.97 0.  ]
